This notebook was used to resample the ground truth images to match the pixel size of Sentinel-1 images and train the Sentinel-1 standalone model. This was necessary because the ground truth was generated using Planet-NICFI images, so it originally had a pixel size of 4.77 m.

In [ ]:
# !pip install rasterio -qqq

In [ ]:
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

In [ ]:
# desired output dimensions
# it was chosen 180x180 because it was the dimension of Sentinel-1 images
target_width = 180
target_height = 180

# path of ground truth images
gt_folder_path = '/content/drive/MyDrive/mgi_thesis/gt_binary'


# iterate over all ground truth images in the folder
for gt_image_filename in os.listdir(gt_folder_path):
    gt_image_path = os.path.join(gt_folder_path, gt_image_filename)
    if gt_image_path.endswith('.tif'):
        with rasterio.open(gt_image_path) as gt_dataset:

            # resample the image
            data = gt_dataset.read(
                out_shape=(gt_dataset.count, target_height, target_width),
                resampling=Resampling.bilinear
            )
            data = np.squeeze(data)

            # Update metadata for the resampled image
            out_meta = gt_dataset.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "width": target_width,
                "height": target_height,
                "count": 1,  # ground truth images have only 1 bands

                # update transform based on new dimensions
                "transform": rasterio.transform.from_bounds(*gt_dataset.bounds, target_width, target_height)
            })

            # save the resampled image
            target_folder = '/content/drive/MyDrive/mgi_thesis/gt_binary_resampled_s1'
            resampled_gt_image_path = os.path.join(target_folder, f"resampled_{gt_image_filename}")
            with rasterio.open(resampled_gt_image_path, 'w', **out_meta) as out_dataset:
                out_dataset.write(data, 1)
